Вы получили основные навыки обработки данных, теперь пора испытать их на практике. Сейчас вам предстоит заняться задачей классификации.

Представлен датасет центра приюта животных, и вашей задачей будет обучить модель таким образом, чтобы  по определенным признакам была возможность максимально уверенно предсказать метки 'Adoption' и 'Transfer' (столбец “outcome_type”).

Здесь вы вольны делать что угодно. Я хочу видеть от вас:
1. Проверка наличия/обработка пропусков
2. Проверьте взаимосвязи между признаками
3. Попробуйте создать свои признаки
4. Удалите лишние
5. Обратите внимание на текстовые столбцы. Подумайте, что можно извлечь полезного оттуда
6. Использование профайлера вам поможет.
7. Не забывайте, что у вас есть PCA (Метод главных компонент). Он может пригодиться.

Вспомните о всем, что я говорил на предыдущих занятиях. Не все будет пригодится, но в жизни вам никто не будет говорить, что использовать :)

Хорошим классификатором для этой задачи будет "Случайный лес" (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

Понимать суть работы "леса" не обязательно на данном этапе, но качество предсказаний будет выше, чем с линейным классификатором. (если желаете, вот гайд https://adataanalyst.com/scikit-learn/linear-classification-method/)

Желаю успеха :)

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import numpy as np

%matplotlib inline

In [97]:
dataset = pd.read_csv('data/aac_shelter_outcomes.csv')
dataset.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown


In [98]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78256 entries, 0 to 78255
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   age_upon_outcome  78248 non-null  object
 1   animal_id         78256 non-null  object
 2   animal_type       78256 non-null  object
 3   breed             78256 non-null  object
 4   color             78256 non-null  object
 5   date_of_birth     78256 non-null  object
 6   datetime          78256 non-null  object
 7   monthyear         78256 non-null  object
 8   name              54370 non-null  object
 9   outcome_subtype   35963 non-null  object
 10  outcome_type      78244 non-null  object
 11  sex_upon_outcome  78254 non-null  object
dtypes: object(12)
memory usage: 3.6+ MB


Всего в датасете 78256 записей. Предположительно наблюдей достаточно для построения модели. 

Есть пропущенные значения в полях: age_upon_outcome, name, outcome_subtype, outcome_type, outcome_type, sex_upon_outcome. Наибольшее количество пропусков в name и outcome_subtype.

На текущий момент все признаки являются категориальными с типом object. Требуетя работа с признаками по отдельности и преобразование. 

In [99]:
dataset.describe()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
count,78248,78256,78256,78256,78256,78256,78256,78256,54370,35963,78244,78254
unique,46,70855,5,2128,525,5869,64361,64361,14574,19,9,5
top,1 year,A706536,Dog,Domestic Shorthair Mix,Black/White,2014-05-05T00:00:00,2016-04-18T00:00:00,2016-04-18T00:00:00,Bella,Partner,Adoption,Neutered Male
freq,14355,11,44242,23335,8153,112,39,39,344,19660,33112,27784


Посмотрим каждый столбец и попробуем преобразовать признаки.

### animal_id

In [100]:
dataset.animal_id.value_counts()

A706536    11
A718223    11
A721033    10
A694501     8
A716018     8
           ..
A714560     1
A727788     1
A700867     1
A675003     1
A711958     1
Name: animal_id, Length: 70855, dtype: int64

AnimalID представляет уникальный индентификатор животного и, очевидно, не влияет на вероятный исход. 

Удалим этот признак. 

In [101]:
dataset = dataset.drop('animal_id', axis=1)

### Визуально складывается ощущение, что столбцы datetime и monthyear имеют одинаковое значение. Проверим эту гипотезу.

In [102]:
dataset['datasetates_check'] = dataset.datetime == dataset.monthyear

In [103]:
dataset.datasetates_check.value_counts()

True    78256
Name: datasetates_check, dtype: int64

Признаки абсолютно идентичны, удалим признак monthyear, а также технический признак datasetates_check.

In [104]:
dataset = dataset.drop(['monthyear','datasetates_check'], axis=1)

### age_upon_outcome

In [105]:
dataset.age_upon_outcome.value_counts()

1 year       14355
2 years      11194
2 months      9213
3 years       5157
3 months      3442
1 month       3344
4 years       2990
5 years       2691
4 months      2425
5 months      1951
6 months      1897
6 years       1810
8 years       1554
7 years       1537
3 weeks       1467
2 weeks       1330
10 months     1204
4 weeks       1194
8 months      1178
10 years      1159
7 months       963
9 years        822
9 months       673
12 years       609
1 weeks        513
11 months      490
11 years       429
1 week         427
13 years       389
14 years       253
3 days         235
2 days         217
15 years       208
1 day          153
6 days         152
4 days         136
5 days         116
16 years       101
0 years         95
5 weeks         61
17 years        58
18 years        26
19 years        13
20 years        12
22 years         4
25 years         1
Name: age_upon_outcome, dtype: int64

Есть много разных значения, причем можно выделить разновидности - дни, недели, месяцы, годы. 

В идеале все привести к какому-то одному значению, возможно в годах. Чтобы это сделать необходимо сначала все пересчитать в количество дней, а затем поделить на 365. 

In [106]:
dataset['value'] = dataset.age_upon_outcome.str.split(' ').str[0].fillna(0).astype(int)
dataset['category'] = dataset.age_upon_outcome.str.split(' ').str[1].fillna(0)

In [107]:
dataset['category'].unique()

array(['weeks', 'year', 'years', 'months', 'month', 'days', 'week', 'day',
       0], dtype=object)

In [108]:
dataset['category'] =   np.where(dataset['category'].str.contains('day'), 1, 
                        np.where(dataset['category'].str.contains('week'), 7, 
                        np.where(dataset['category'].str.contains('month'), 30, 
                        np.where(dataset['category'].str.contains('year'), 365, 0)))).astype(int)

dataset['outcome_age_(days)'] = dataset['category'] * dataset['value']
dataset['outcome_age_(years)'] = dataset['outcome_age_(days)'] / 365

In [109]:
dataset = dataset.drop(['age_upon_outcome','value','category','outcome_age_(days)'], axis=1)

In [110]:
dataset['outcome_age_(years)'].count()

78256

In [144]:
dataset['outcome_age_(years)'].value_counts().count()

45

На 78 256 записей получается 45 уникальных записей о возврасте. Далее преобразования не производим. 

Предположим, что помимо самого показателя возраста особо влияение оказывает молодость (щенки, котята до 0,5 года) или наоборот старший возраст (более 10 лет). Так как не решать планируем при помощи дерева решений, то oh_encoding не применяем, только определяем категории. 

In [116]:
dataset['old/young'] =  np.where(dataset['outcome_age_(years)'] <= 0.5, 'young', 
                        np.where(dataset['outcome_age_(years)'] <= 5, 'medium',
                        np.where(dataset['outcome_age_(years)'] <= 10, 'old', 'very_old')))

In [118]:
dataset['old/young'].value_counts()

medium      40895
young       28376
old          6882
very_old     2103
Name: old/young, dtype: int64

### name

In [127]:
dataset['name'] = dataset['name'].str.strip('*')

In [128]:
dataset.name.value_counts()

Bella       367
Max         367
Charlie     291
Daisy       285
Luna        274
           ... 
Nimbus        1
Mauve         1
Anisette      1
Miette        1
Dolo          1
Name: name, Length: 11790, dtype: int64

In [129]:
names_freq_df = dataset.name.value_counts().rename_axis('unique_values').reset_index(name='name_freq')

In [130]:
names_freq_df

,unique_values,name_freq
0,Bella,367
1,Max,367
2,Charlie,291
3,Daisy,285
4,Luna,274
...,...,...
11785,Nimbus,1
11786,Mauve,1
11787,Anisette,1
11788,Miette,1


In [131]:
dataset = dataset.merge(names_freq_df, left_on='name', right_on='unique_values')

Предположим, что само имя для нас не имеет значимости, но животных с наиболее популярными именами могут забирать из приюта чаще. 

Присвоим категории частоты имени.

In [136]:
dataset['popular_name'] =   np.where(dataset['name_freq'] >= 150, 'very popular', 
                            np.where(dataset['name_freq'] >= 50, 'popular',
                            np.where(dataset['name_freq'] > 0, 'not popular', 'no name')))

In [138]:
dataset = dataset.drop(['name','unique_values','name_freq'], axis=1)

### date_of_birth & datetime

Приведем столбцы к формату времени datetime

In [139]:
dataset

,animal_type,breed,color,date_of_birth,datetime,outcome_subtype,outcome_type,sex_upon_outcome,outcome_age_(years),old/young,pupular_name
0,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,Partner,Transfer,Spayed Female,1.000000,medium,very popular
1,Cat,Domestic Shorthair Mix,Blue Tabby/White,2014-06-16T00:00:00,2014-08-14T18:45:00,NaN,Adoption,Intact Female,0.082192,young,very popular
2,Cat,Domestic Longhair Mix,Brown Tabby,2014-10-01T00:00:00,2014-12-13T14:16:00,Foster,Adoption,Spayed Female,0.164384,young,very popular
3,Dog,Maltese Mix,White,2000-11-16T00:00:00,2013-11-16T13:31:00,NaN,Return to Owner,Spayed Female,13.000000,very_old,very popular
4,Dog,Labrador Retriever Mix,Yellow/White,2013-11-13T00:00:00,2014-11-30T14:21:00,Partner,Transfer,Spayed Female,1.000000,medium,very popular
...,...,...,...,...,...,...,...,...,...,...,...
54365,Dog,Labrador Retriever,Tan,2005-04-28T00:00:00,2018-02-01T13:17:00,Foster,Adoption,Spayed Female,12.000000,very_old,not popular
54366,Dog,Jack Russell Terrier Mix,White,2016-01-02T00:00:00,2018-01-04T12:43:00,NaN,Return to Owner,Intact Male,2.000000,medium,not popular
54367,Cat,Domestic Medium Hair Mix,Brown Tabby/White,2016-07-29T00:00:00,2018-02-01T14:50:00,NaN,Adoption,Spayed Female,1.000000,medium,not popular
54368,Dog,Labrador Retriever Mix,Tan/White,2015-12-11T00:00:00,2018-02-01T15:13:00,NaN,Adoption,Spayed Female,2.000000,medium,not popular


In [ ]:
нормализацию для дерева решений не нужно